<a href="https://colab.research.google.com/github/TrybinD/educational-projects/blob/main/educational-projects/ML%20projects/Titanic_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"daniiltrubin","key":"a0e37acddb328217e1f38b5525aa2e78"}'}

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c titanic -q

Здесь начинается работа с датасетом

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_train = pd.read_csv("train.csv", index_col="PassengerId")
data_test = pd.read_csv("test.csv", index_col="PassengerId")

In [ ]:
data_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
X, y = data_train.drop(["Survived"], axis=1), data_train.Survived

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 76.6+ KB


In [ ]:
drop_cols = ["Name", "Ticket", "Cabin", "SibSp", "Parch"]
cat_cols = ["Sex", "Embarked", "Pclass"]
num_cols = ["Age",  "Fare", "SibSp", "Parch"]

In [ ]:
X[drop_cols]

,Name,Ticket,Cabin,SibSp,Parch
PassengerId,,,,,
1,"Braund, Mr. Owen Harris",A/5 21171,NaN,1,0
2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",PC 17599,C85,1,0
3,"Heikkinen, Miss. Laina",STON/O2. 3101282,NaN,0,0
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",113803,C123,1,0
5,"Allen, Mr. William Henry",373450,NaN,0,0
...,...,...,...,...,...
887,"Montvila, Rev. Juozas",211536,NaN,0,0
888,"Graham, Miss. Margaret Edith",112053,B42,0,0
889,"Johnston, Miss. Catherine Helen ""Carrie""",W./C. 6607,NaN,1,2


In [ ]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer

In [ ]:
def selector(atribute_names):
  return FunctionTransformer(lambda x: x[atribute_names])

In [ ]:
def split_names(x):
  ans = []
  ans2 = []
  for i in x:
    ans.append(i.split(', ')[1].split('.')[0])
  for n in ans:
    ans2.append(n if n in ["Mr", "Miss", "Mrs", "Master"] else "Other")
  return np.array(ans2).reshape(len(x), 1)


In [ ]:
name_encoder = FunctionTransformer(split_names)

In [ ]:
name_encoder.transform(X.Name)

In [ ]:
X

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
PassengerId,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,0
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,0
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1,1
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,1
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,4,0


In [ ]:
name_pipeline = Pipeline([
                          ("selector", selector("Name")),
                          ('split', name_encoder),
                          ("encoder", OneHotEncoder()) #только самые распостраненные
])

In [ ]:
name_pipeline.fit_transform(X)

<891x5 sparse matrix of type '<class 'numpy.float64'>'
	with 891 stored elements in Compressed Sparse Row format>

In [ ]:
def family_info(X):
  X["FamilySize"] = X['SibSp'] + X["Parch"] + 1
  X['IsAlone'] = 1
  X['IsAlone'].loc[X['FamilySize'] > 1] = 0
  return X[["FamilySize", "IsAlone"]]

In [ ]:
family_pipeline = FunctionTransformer(family_info)

In [ ]:
family_pipeline.fit_transform(X)

In [ ]:
num_pipeline = Pipeline([
    ("selector", selector(num_cols)),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [ ]:
cat_pipeline = Pipeline([
    ('selector', selector(cat_cols)),
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder(sparse=False))                   
])

In [ ]:
full_pipeline = FeatureUnion(transformer_list=[
    ("num", num_pipeline),
    ("cat", cat_pipeline)
])

In [ ]:
X_preprocesed = full_pipeline.fit_transform(X)

In [ ]:
X_preprocesed

array([[-0.56573646, -0.50244517,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.66386103,  0.78684529,  0.43279337, ...,  1.        ,
         0.        ,  0.        ],
       [-0.25833709, -0.48885426, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.1046374 , -0.17626324,  0.43279337, ...,  0.        ,
         0.        ,  1.        ],
       [-0.25833709, -0.04438104, -0.4745452 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.20276197, -0.49237783, -0.4745452 , ...,  0.        ,
         0.        ,  1.        ]])

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
svm_clf = SVC(probability=True)
ada_clf = AdaBoostClassifier()
bag_clf = BaggingClassifier()
rf_clf = RandomForestClassifier()

In [ ]:
cross_val_score(svm_clf, X_preprocesed, y, scoring="accuracy", cv=5)

array([0.84357542, 0.8258427 , 0.82022472, 0.80898876, 0.87640449])

In [ ]:
cross_val_score(rf_clf, X_preprocesed, y, scoring="accuracy", cv=5)

array([0.7877095 , 0.7752809 , 0.84831461, 0.76404494, 0.80898876])

In [ ]:
cross_val_score(ada_clf, X_preprocesed, y, scoring="accuracy", cv=5)

array([0.7877095 , 0.80898876, 0.81460674, 0.81460674, 0.83707865])

In [ ]:
params = {
        'C':[0.1, 1, 2, 5],
        'kernel': ['rbf', 'poly', 'sigmoid'],
        'degree': [2, 3, 5],
        "probability":[True, False],
        "tol": [0.001, 0.01, 0.1, 1]
}

grid_svc = GridSearchCV(svm_clf, params, scoring="accuracy", cv=5)
grid_svc.fit(X_preprocesed,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 2, 5], 'degree': [2, 3, 5],
                         'kernel': ['rbf', 'poly', 'sigmoid'],
                         'probability': [True, False],
                         'tol': [0.001, 0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
grid_svc.best_estimator_

SVC(C=5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.1,
    verbose=False)

In [ ]:
grid_svc.best_score_

0.8338585148452703

In [ ]:
params = {
    'n_estimators': [50, 100, 150, 200],
    "criterion": ["gini",  "entropy"],
    'max_depth': [None, 10, 20, 100],
    'min_samples_split': [2, 3, 4, 5]
}

grid_rf = GridSearchCV(rf_clf, params, scoring="accuracy", cv=5)
grid_rf.fit(X_preprocesed,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
grid_rf.best_estimator_, grid_rf.best_score_

(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=20, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=5,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False), 0.8339087314041805)

In [ ]:
!pip install catboost

     |████████████████████████████████| 69.2 MB 4.8 kB/s 


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat_clf = CatBoostClassifier(verbose=False)

In [ ]:
cat_clf.fit(X_preprocesed, y, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
cross_val_score(cat_clf, X_preprocesed, y, scoring='accuracy', cv=5)

array([0.83240223, 0.80898876, 0.85955056, 0.82022472, 0.84269663])

In [ ]:
best_model = grid_rf.best_estimator_

In [ ]:
best_model.fit(X_preprocesed, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(8)

In [ ]:
X_preprocesed2.shape

(891, 8)

Пишем свои собственные предсказатели

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [ ]:
class StratRandomClassifier(BaseEstimator, ClassifierMixin):

  def __init__(self, n=None):
    self.n = n

  def fit(self, X, y):
    self.proba_dict = (X[y==1][self.n].value_counts()/X[self.n].value_counts()).to_dict()
    return self

  def predict(self, X):
    y = []
    for i in X.values:
      sex = i[self.n]
      y.append(np.random.choice(2, p=[1-self.proba_dict[sex], self.proba_dict[sex]]))
    return np.array(y)

In [ ]:
n=2
(X[y==1][n].value_counts()/X[n].value_counts())

KeyError: ignored

In [ ]:
by_sex_clf = StratRandomClassifier(n=2)
by_Pclass_clf = StratRandomClassifier("Pclass")

In [ ]:
by_sex_clf.fit(X,y)

StratRandomClassifier(atr='Sex', n=2)

In [ ]:
by_sex_clf.predict(X)

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,

In [ ]:
cross_val_score(by_sex_clf, X, y, cv=5)

array([0.65363128, 0.67977528, 0.67977528, 0.6741573 , 0.65730337])

In [ ]:
cross_val_score(by_Pclass_clf, X, y, cv=5)

array([0.51955307, 0.51123596, 0.63483146, 0.54494382, 0.56741573])

In [ ]:
X

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [ ]:
clfs = [svm_clf, rf_clf, ada_clf, bag_clf]

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
X

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [ ]:
X_preprocesed[0]

array([-0.56573646, -0.50244517,  0.43279337, -0.47367361,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  1.        ])

In [ ]:
cat_cols

['Sex', 'Embarked', 'Pclass']

In [ ]:
vot_clf = VotingClassifier(estimators=[("SVM", svm_clf),
                            ("rf", rf_clf),
                            ('cat', cat_clf), ("ada", ada_clf), ("bag", bag_clf), ("by_sex", StratRandomClassifier(n=5))], voting="hard")

In [ ]:
vot_clf.fit(X_preprocesed, y)

AttributeError: ignored

In [ ]:
cross_val_score(vot_clf, X_preprocesed, y, scoring="accuracy", cv=5)

array([0.83240223, 0.8258427 , 0.83707865, 0.80898876, 0.85393258])

In [ ]:
data_test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [ ]:
X_test = full_pipeline.transform(data_test)

In [ ]:
y_pred = vot_clf.predict(X_test)

In [ ]:
y_pred = by_sex_clf.predict(data_test)

In [ ]:
y_pred

array([0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,

In [ ]:
ans = pd.DataFrame({"PassengerId": [i for i in range(892, 1310)],"Survived": y_pred})

In [ ]:
ans.to_csv("submission_my_clf.csv", index=False)

In [ ]:
!kaggle competitions submit -c titanic -f submission_my_clf.csv -m "Message"

100% 2.77k/2.77k [00:08<00:00, 317B/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [ ]:
dd = np.array([1,2,3])

In [ ]:
dd.